In [ ]:
import batoid
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib widget

In [ ]:
def colorbar(mappable, **kwargs):
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    import matplotlib.pyplot as plt
    last_axes = plt.gca()
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    cbar = fig.colorbar(mappable, cax=cax, **kwargs)
    plt.sca(last_axes)
    return cbar

In [ ]:
telescope = batoid.Optic.fromYaml("LSST_g.yaml")
scale = 0.2/10e-6  # arcsec / meter

In [ ]:
# Make refractive interfaces partially reflective
reflect = 0.02
transmit = 0.98
for surface in telescope.itemDict.values():
    if isinstance(surface, batoid.RefractiveInterface):
        surface.forwardCoating = batoid.SimpleCoating(reflect, transmit)
        surface.reverseCoating = batoid.SimpleCoating(reflect, transmit)
    if isinstance(surface, batoid.Detector):
        surface.forwardCoating = batoid.SimpleCoating(reflect, transmit)
# Allow filter entrance surface to be trickier
telescope['Filter_entrance'].forwardCoating = batoid.SimpleCoating(0.5, 0.5)

In [ ]:
angle = 1.0
dirCos = batoid.utils.gnomonicToDirCos(0.0, np.deg2rad(angle))

rays = batoid.RayVector.asPolar(
    optic=telescope, wavelength=500e-9,
    theta_x=0.0, theta_y=np.deg2rad(angle),
    # nrad=300, naz=900
    nrandom=int(1e5)
)

rForward, rReverse = telescope.traceSplit(rays, minFlux=1e-6)

In [ ]:
print("# input rays          = {}".format(len(rays)))
print("# forward output rays = {}".format(sum(len(rr) for rr in rForward)))
print("# reverse output rays = {}".format(sum(len(rr) for rr in rReverse)))
print("input flux          = {}".format(np.sum(rays.flux)))
forwardFlux = np.sum([np.sum(rr.flux) for rr in rForward])
reverseFlux = np.sum([np.sum(rr.flux) for rr in rReverse])
print("forward output flux = {}".format(forwardFlux))
print("reverse output flux = {}".format(reverseFlux))
print("destroyed flux      = {}".format(
    np.sum(rays.flux) - forwardFlux - reverseFlux
))

In [ ]:
# Separate out the direct path
direct_path_idx = np.argmin([len(r.path) for r in rForward])
direct_path = rForward[direct_path_idx]
del rForward[direct_path_idx]  

In [ ]:
direct_flux = np.sum(direct_path.flux)

In [ ]:
x = np.concatenate([rr.x for rr in rForward])
y = np.concatenate([rr.y for rr in rForward])
flux = np.concatenate([rr.flux for rr in rForward])

In [ ]:
H, xedges, yedges = np.histogram2d(
    x, y, 
    weights=flux, 
    bins=200, 
    range=[[-0.38, 0.38], [-0.38, 0.38]]
)

In [ ]:
# What is the area of each pixel?
dx = np.mean(np.diff(xedges))
dy = np.mean(np.diff(yedges))
area = dx*dy # in m^2
area *= (1/10e-6)**2 # pix^2
area *= (1/0.2)**2 # arcsec^2
print(area)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 4))
colorbar(ax.imshow(
    -2.5*np.log10((H.T+1e-300)/area/direct_flux), 
    vmin=25, vmax=33,
    cmap='viridis_r',
    extent=[-0.38, 0.38, -0.38, 0.38]
), label='mag/sq arcsec')
ax.set_aspect(1)
ax.set_title("0th magnitude ghosts")
plt.show()